Multi-Class Prediction of Obesity Risk
----

www.kaggle.com/competitions/playground-series-s4e2/overview/

Results will be evaluated using the accuracy score.

This notebook can be run in Colab as well, with automatically generated code to load the linked data.

# Setup

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import islice
import matplotlib as mpl
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')

print("Output format:\n---------------")
with open('/kaggle/input/playground-series-s4e2/sample_submission.csv') as f:
    for line in islice(f, 10): print(line)


Output format:
---------------
id,NObeyesdad

20758,Normal_Weight

20759,Normal_Weight

20760,Normal_Weight

20761,Normal_Weight

20762,Normal_Weight

20763,Normal_Weight

20764,Normal_Weight

20765,Normal_Weight

20766,Normal_Weight



# EDA

In [3]:
print(train_data.info())
print()
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [4]:
eda_data = pd.concat([train_data, test_data])

In [5]:
eda_data.head().T

,0,1,2,3,4
id,0,1,2,3,4
Gender,Male,Female,Female,Female,Male
Age,24.443011,18.0,18.0,20.952737,31.641081
Height,1.699998,1.56,1.71146,1.71073,1.914186
Weight,81.66995,57.0,50.165754,131.274851,93.798055
family_history_with_overweight,yes,yes,yes,yes,yes
FAVC,yes,yes,yes,yes,yes
FCVC,2.0,2.0,1.880534,3.0,2.679664
NCP,2.983297,3.0,1.411685,3.0,1.971472
CAEC,Sometimes,Frequently,Sometimes,Sometimes,Sometimes


## Continous measures

In [6]:
eda_data.drop(columns=['id']).describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000,34598.000000
mean,23.886181,1.699721,87.686451,2.444704,2.757043,2.030469,0.978861,0.614467
std,5.733207,0.087895,26.273493,0.532568,0.707610,0.609566,0.839122,0.604475
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,20.000000,1.631856,66.000000,2.000000,3.000000,1.784710,0.006892,0.000000
50%,22.851747,1.700000,84.000000,2.392179,3.000000,2.000000,1.000000,0.555591
75%,26.000000,1.761773,111.539494,3.000000,3.000000,2.550570,1.583832,1.000000
max,61.000000,1.980000,165.057269,3.000000,4.000000,3.000000,3.000000,2.000000


These are all relatively centered, albeit with different variances so they will need to be standardized.

## Categorical measures

In [7]:
cat_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS', 'NObeyesdad']

for col in cat_cols:
    print(eda_data[col].value_counts())
    print('------------')

Gender
Female    17387
Male      17211
Name: count, dtype: int64
------------
family_history_with_overweight
yes    28398
no      6200
Name: count, dtype: int64
------------
FAVC
yes    31565
no      3033
Name: count, dtype: int64
------------
CAEC
Sometimes     29218
Frequently     4089
Always          837
no              454
Name: count, dtype: int64
------------
SMOKE
no     34173
yes      425
Name: count, dtype: int64
------------
SCC
no     33447
yes     1151
Name: count, dtype: int64
------------
CALC
Sometimes     25045
no             8676
Frequently      875
Always            2
Name: count, dtype: int64
------------
MTRANS
Public_Transportation    27798
Automobile                5939
Walking                    747
Motorbike                   57
Bike                        57
Name: count, dtype: int64
------------
NObeyesdad
Obesity_Type_III       4046
Obesity_Type_II        3248
Normal_Weight          3082
Obesity_Type_I         2910
Insufficient_Weight    2523
Overweight_Level

The targets are largely balanced.

Note that while the labels are categorical, there are ordinal: an individual who is more likely to be obese relative to normal weight should also be more likely to be overweight relative to normal weight.

In [8]:
target_categories_order = [
    'Insufficient_Weight',
    'Normal_Weight',
    'Overweight_Level_I',
    'Overweight_Level_II',
    'Obesity_Type_I',
    'Obesity_Type_II',
    'Obesity_Type_III'
]

eda_data['target_ord'] = pd.Categorical(eda_data.NObeyesdad, target_categories_order, ordered=True)

## Binary associations